In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import gridspec
import time

In [2]:
from matplotlib import rc
rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

mpl.rcParams.update({'font.size': 19})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

import matplotlib.pyplot as plt
import mplhep as hep
hep.set_style("CMS") 

<ipython-input-2-99a23e5daf14>:21: FutureWarning: ``set_style`` is deprecated: Naming convention is changing to match mpl. Use ``mplhep.style.use()``.
  hep.set_style("CMS")


### New Binning for pT (only retaining 4 bins for low Q^2)

In [4]:
qedcorr_rapgap = {}
qedcorr_djangoh = {} 

with open('outputfiles/QEDCorrections_Rapgap.npy' , 'rb') as f:
    qedcorr_rapgap['pT'] = np.load(f)[0:4]
    qedcorr_rapgap['eta'] = np.load(f)
    qedcorr_rapgap['qT'] = np.load(f)
    qedcorr_rapgap['dphi'] = np.load(f)
with open('outputfiles/QEDCorrections_Djangoh.npy' , 'rb') as f:
    qedcorr_djangoh['pT'] = np.load(f)[0:4]
    qedcorr_djangoh['eta'] = np.load(f)
    qedcorr_djangoh['qT'] = np.load(f)
    qedcorr_djangoh['dphi'] = np.load(f)
    

mc = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Django_nominal.pkl")
theta0_G = mc[['genjet_pt','genjet_eta','genjet_dphi','genjet_qtnorm','gen_y_sigma']].to_numpy()
weights_MC = mc['wgt']
pass_fid = np.array(mc['pass_fiducial'])
y = theta0_G[:,4][pass_fid==1]

del mc
gc.collect()

cuts = np.load("/global/home/users/yxu2/disjets/FinalReading/Qbinned_result/inputfiles/y_cuts.npy")
file_loc = "storage_files"


symbol = {}
symbol['dphi'] = '$\Delta\phi^{jet}$ [rad]'
symbol['eta'] = '$\eta^{jet}$'
symbol['qT']  = '$q_{T}^{jet}/Q$'
symbol['pT'] = '$p_{T}^{jet}$ [GeV]'

bins = {}

#jet pt
#bins['pT'] = np.load("inputfiles/pT_bin.npy")
bins['pT'] = np.logspace(np.log10(10),np.log10(100),7)
#bins['pT'] = np.array([10.0, 14.677992676220699, 21.544346900318832, 31.622776601683793, 46.41588833612777, 68.12920690579611, 100.0])


#jet eta
bins['eta'] = np.linspace(-1,2.5,6)

#dphi
bins['dphi'] = np.logspace(np.log10(0.03),np.log10(np.pi/2.0),9) - 0.03
bins['dphi'] = bins['dphi'][1:]
bins['dphi'][0] = 0.0

#qt
bins['qT'] = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
bins['qT'] = bins['qT'][1:]
bins['qT'][0] = 0.0
for k in range(len(cuts)-1):
    y_cut = np.where((cuts[k] < y) & (y < cuts[k+1]), True, False)
    cut_label = str(cuts[k])+" < y < "+str(cuts[k+1]) #+ " (GeV^2)"
    
    truthval = {}
    truthval["pT"],_=np.histogram(theta0_G[pass_fid==1][:,0][y_cut],bins=bins['pT'],weights=weights_MC[pass_fid==1][y_cut],density=True)
    truthval["qT"],_=np.histogram(theta0_G[pass_fid==1][:,3][y_cut],bins=bins["qT"],weights=weights_MC[pass_fid==1][y_cut],density=True)
    truthval["dphi"],_=np.histogram(theta0_G[pass_fid==1][:,2][y_cut],bins=bins["dphi"],weights=weights_MC[pass_fid==1][y_cut],density=True)
    truthval["eta"],_=np.histogram(theta0_G[pass_fid==1][:,1][y_cut],bins=bins["eta"],weights=weights_MC[pass_fid==1][y_cut],density=True)
    

    
    for obs in ['pT','qT','dphi','eta']:

        nominal_R = []
        sys0_R = []
        sys1_R = []
        sys5_R = []
        sys7_R = []
        sys11_R = []

        nominal_D = []
        sys0_D = []
        sys1_D = []
        sys5_D = []
        sys7_D = []
        sys11_D = []


        for i in range(8):

            nominal_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys0_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_0n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys1_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_1n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys5_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_5n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys7_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_7n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys11_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_11n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]

            nominal_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_nominaln_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys0_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_0n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys1_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_1n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys5_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_5n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys7_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_7n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            sys11_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_11n_Omni_step2_"+obs+"_iteration4"+cut_label+".npy")]
            pass

        #non-closure
        nc = np.load("inputfiles/nonclosure/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4_NC_"+cut_label+".npy")
        nc = (nc-truthval[obs])/truthval[obs]

        #stat uncert
        nominal_stat = []
        for j in np.concatenate([range(1,30),range(34,45),[46,47,48,49],range(54,66),range(80,86),range(100,106),range(120,126)]):
            nominal_stat += [np.load("inputfiles/fullscan_stat/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4_"+str(j)+cut_label+".npy")]
            pass

        nominal_stat = nominal_stat[0:74]
        nominal_stat = np.reshape(nominal_stat,[74,1,np.shape(nominal_stat)[-1]])
        nominal_stat = np.median(nominal_stat,axis=1)


        fig = plt.figure()

        nominal_rap = np.median(nominal_R,axis=0)

        sys0_rap = np.median(sys0_R,axis=0)
        sys1_rap = np.median(sys1_R,axis=0)
        sys5_rap = np.median(sys5_R,axis=0)
        sys7_rap = np.median(sys7_R,axis=0)
        sys11_rap = np.median(sys11_R,axis=0)

        #std
        std_nom_R = np.std(nominal_R,axis=0)/np.median(nominal_R,axis=0)
        std_sys0_R = np.std(sys0_R,axis=0)/np.median(sys0_R,axis=0)
        std_sys1_R = np.std(sys1_R,axis=0)/np.median(sys1_R,axis=0)
        std_sys5_R = np.std(sys5_R,axis=0)/np.median(sys5_R,axis=0)
        std_sys7_R = np.std(sys7_R,axis=0)/np.median(sys7_R,axis=0)
        std_sys11_R = np.std(sys11_R,axis=0)/np.median(sys11_R,axis=0)
        std_nom_D = np.std(nominal_D,axis=0)/np.median(nominal_D,axis=0)
        std_sys0_D = np.std(sys0_D,axis=0)/np.median(sys0_D,axis=0)
        std_sys1_D = np.std(sys1_D,axis=0)/np.median(sys1_D,axis=0)
        std_sys5_D = np.std(sys5_D,axis=0)/np.median(sys5_D,axis=0)
        std_sys7_D = np.std(sys7_D,axis=0)/np.median(sys7_D,axis=0)
        std_sys11_D = np.std(sys11_D,axis=0)/np.median(sys11_D,axis=0)
        std = np.mean([std_nom_R,std_sys0_R,std_sys1_R,std_sys5_R,std_sys7_R,std_sys11_R,
                      std_nom_D,std_sys0_D,std_sys1_D,std_sys5_D,std_sys7_D,std_sys11_D],axis=0)
        stat = np.std(nominal_stat,axis=0)/np.mean(nominal_stat,axis=0)
        stat = np.sqrt(abs(stat**2-std**2))
  

        
        nominal_dj = np.median(nominal_D,axis=0)
        sys0_dj = np.median(sys0_D,axis=0)
        sys1_dj = np.median(sys1_D,axis=0)
        sys5_dj = np.median(sys5_D,axis=0)
        sys7_dj = np.median(sys7_D,axis=0)
        sys11_dj = np.median(sys11_D,axis=0)

        myavgR_0 = (nominal_rap - sys0_rap)/nominal_rap
        myavgR_1 = (nominal_rap - sys1_rap)/nominal_rap
        myavgR_5 = (nominal_rap - sys5_rap)/nominal_rap
        myavgR_7 = (nominal_rap - sys7_rap)/nominal_rap
        myavgR_11 = (nominal_rap - sys11_rap)/nominal_rap

        myavgD_0 = (nominal_dj - sys0_dj)/nominal_dj
        myavgD_1 = (nominal_dj - sys1_dj)/nominal_dj
        myavgD_5 = (nominal_dj - sys5_dj)/nominal_dj
        myavgD_7 = (nominal_dj - sys7_dj)/nominal_dj
        myavgD_11 = (nominal_dj - sys11_dj)/nominal_dj

        myavg_0 = np.mean([myavgR_0,myavgD_0],axis=0)
        myavg_1 = np.mean([myavgR_1,myavgD_1],axis=0)
        myavg_5 = np.mean([myavgR_5,myavgD_5],axis=0)
        myavg_7 = np.mean([myavgR_7,myavgD_7],axis=0)
        myavg_11 = np.mean([myavgR_11,myavgD_11],axis=0)

        model_nominal = (nominal_rap-nominal_dj)/nominal_rap
        model_0 = (sys0_rap-sys0_dj)/sys0_rap
        model_1 = (sys1_rap-sys1_dj)/sys1_rap
        model_5 = (sys5_rap-sys5_dj)/sys5_rap
        model_7 = (sys7_rap-sys7_dj)/sys7_rap
        model_11 = (sys11_rap-sys11_dj)/sys11_rap
        model = np.mean([model_nominal,model_0,model_1,model_5,model_7,model_11],axis=0)
        qed = (qedcorr_rapgap[obs]-qedcorr_djangoh[obs])/qedcorr_rapgap[obs]

        fig = plt.figure(figsize=(7, 5)) 
        gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
        ax0 = plt.subplot(gs[0])
        ax0.yaxis.set_ticks_position('both')
        ax0.xaxis.set_ticks_position('both')
        ax0.tick_params(direction="in",which="both")
        ax0.minorticks_on()
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)

        xvals = 0.5*(bins[obs][0:-1]+bins[obs][1:])

        #plt.plot(xvals, 100*abs(qed),label="QED rad. corr.",color="tab:blue")
        plt.plot(xvals, 100*abs(nc),label="Non-closure",color="tab:orange")
        plt.plot(xvals,100*abs(myavg_0),label="HFS scale (in jet)",color="tab:green")
        plt.plot(xvals,100*abs(myavg_1),label="HFS scale (remainder)",color="tab:red")
        plt.plot(xvals,100*abs(myavg_5),label="HFS $\phi$ angle",color="tab:purple")
        plt.plot(xvals,100*abs(myavg_7),label="Lepton energy scale",color="tab:brown")
        plt.plot(xvals,100*abs(myavg_11),label="Lepton $\phi$ angle",color="tab:pink")
        plt.plot(xvals,100*abs(model),label="Model",color="tab:gray")
        plt.plot(xvals,100*abs(stat),label="Stat.",ls="--",lw=3,color="tab:olive")

        #total = np.sqrt(nc**2+qed**2+myavg_0**2+myavg_1**2+myavg_5**2+myavg_7**2+myavg_11**2+model**2)
        #turned off QED uncertainty
        total = np.sqrt(nc**2+myavg_0**2+myavg_1**2+myavg_5**2+myavg_7**2+myavg_11**2+model**2)
        plt.plot(xvals,100*abs(total),ls=":",color="black",label="Total Syst.",lw=3)

        plt.legend(loc='upper right',fontsize=14,ncol=2,bbox_to_anchor=(0.96, 0.9))
        plt.ylim([0,32])
        plt.xlabel(symbol[obs],fontsize=20)
        plt.ylabel("Systematic Uncertainty [%]",fontsize=20)

        if obs == 'qT':
            plt.xscale("log")

        cut_label2 = str(cuts[k])+" $< y <$ "+str(cuts[k+1]) #+ " (GeV^2)"
        plt.text(0.1, 1.15-0.25,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
        plt.text(0.6, 1.15-0.25, '$Q^{2}>$150 GeV$^{2}$, '+cut_label2+', $p_{T}^{jet}>10$ GeV', horizontalalignment='center',verticalalignment='center', transform = ax0.transAxes, fontsize=13)

        myobs_map = {}
        myobs_map['pT'] = 'jetpt'
        myobs_map['eta'] = 'jeteta'
        myobs_map['qT'] = 'qt'
        myobs_map['dphi'] = 'dphi'

        fig.savefig('figures/TotalUncerts_%s_FIXED_%s.png'%(myobs_map[obs],cut_label),bbox_inches='tight')

        #save results
        with open('outputfiles/ResultwithSystematicUncertainties_%s_ensemble_FIXED_%s.npy'%(myobs_map[obs],cut_label), 'wb') as f:
            np.save(f,0.5*(bins[obs][0:-1]+bins[obs][1:]))
            np.save(f,np.mean([nominal_rap,nominal_dj],axis=0))
            np.save(f,0.5*abs(bins[obs][0:-1]-bins[obs][1:]))
            np.save(f, total)
            np.save(f, stat)
            np.save(f,abs(qed))
            np.save(f,abs(myavg_0))
            np.save(f,abs(myavg_1))
            np.save(f,abs(myavg_5))
            np.save(f,abs(myavg_7))
            np.save(f,abs(myavg_11))
            np.save(f,abs(model))
            
del theta0_G,weights_MC,pass_fid
gc.collect()

<ipython-input-4-af15be6313a3>:116: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


9071